**Bài toán 2: Tìm tập phổ biến cho bài toán đề xuất phim**. Dữ liệu lấy tại 
https://grouplens.org/datasets/movielens với MovieLens 100K dataset 

**Nhiệm vụ 1:** Tìm tập phổ biến có trong tập dữ liệu phim 

1. Import các gói thư viện, nạp dữ liệu và tiền xử lý dữ liệu vào notebook

In [2]:
import os 
import pandas as pd 
data_folder = "Dataset/ml-100k" 
ratings_filename = os.path.join(data_folder, "u.data") 
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, 
  names = ["UserID", "MovieID", "Rating", "Datetime"]) 
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],  
 unit='s') 
# Tạo cột mới tên Favorable 
all_ratings["Favorable"] = all_ratings["Rating"] > 3 
ratings = all_ratings[all_ratings['UserID'].isin(range(200))] 
favorable_ratings = ratings[ratings["Favorable"]] 
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in 
  favorable_ratings.groupby("UserID")["MovieID"]) 
num_favorable_by_movie = ratings[["MovieID",  
 "Favorable"]].groupby("MovieID").sum() 
num_favorable_by_movie.sort_values(by="Favorable", 
ascending=False).head()

,Favorable
MovieID,
50,100
100,89
258,83
181,79
174,74


2.Tạo hàm để tìm tập phổ biến

In [3]:
from collections import defaultdict 
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, 
min_support): 
    counts = defaultdict(int) 
    for user, reviews in favorable_reviews_by_users.items(): 
        for itemset in k_1_itemsets: 
            if itemset.issubset(reviews): 
                for other_reviewed_movie in reviews - itemset: 
                    current_superset = itemset | frozenset((other_reviewed_movie,)) 
                    counts[current_superset] += 1 
    return dict([(itemset, frequency) for itemset, frequency in 
                 counts.items() if frequency >= min_support]) 

3.Tìm tập phổ biến

In [4]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length 
min_support = 50 
# k=1 candidates are the isbns with more than min_support favourable reviews 
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"]) 
            for movie_id, row in num_favorable_by_movie.iterrows() 
                if row["Favorable"] > min_support) 
print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support)) 

sys.stdout.flush() 
for k in range(2, 20): 
# Generate candidates of length k, using the frequent itemsets of length k-1 
# Only store the frequent itemsets 
    cur_frequent_itemsets =  find_frequent_itemsets(favorable_reviews_by_users, 
                frequent_itemsets[k-1], min_support) 
    if len(cur_frequent_itemsets) == 0: 
        print("Did not find any frequent itemsets of length {}".format(k)) 
        sys.stdout.flush() 
        break
    else:    
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k)) 
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets 

# We aren't interested in the itemsets of length 1, so remove those 
del frequent_itemsets[1] 

There are 16 movies with more than 50 favorable reviews
I found 93 frequent itemsets of length 2
I found 295 frequent itemsets of length 3
I found 593 frequent itemsets of length 4
I found 785 frequent itemsets of length 5
I found 677 frequent itemsets of length 6
I found 373 frequent itemsets of length 7
I found 126 frequent itemsets of length 8
I found 24 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11
